In [ ]:
%matplotlib inline
%env TQDM_DISABLE=1
import warnings
warnings.filterwarnings('ignore')
# tag: remove-cell applied

# Batch Execution

If you have several dose-response datasets, you can run a batch.  As an example, consider a CSV or Excel file of dichotomous datasets that looks something like this (with one row per dataset):

| ID | Dose              | Incidence | N         |
|----|-------------------|-----------|-----------|
| 1  | 0;0.5;1           | 0;3;5     | 5;5;5     |
| 2  | 0;0.33;0.67;1     | 0;0;4;5   | 5;5;5;5   |
| 3  | 0;0.25;0.5;1      | 0;0;3;5   | 5;5;5;5   |
| 4  | 0;0.33;0.67;1     | 0;0;1;1   | 5;5;5;5   |
| 5  | 0;0.25;0.5;1      | 0;0;1;1   | 5;5;5;5   |
| 6  | 0;0.33;0.67;1     | 0;0;1;1   | 5;5;5;5   |
| 7  | 0;0.25;0.5;1      | 0;0;1;1   | 5;5;5;5   |
| 8  | 0;0.25;0.5;0.75;1 | 0;0;1;3;1 | 5;5;5;5;5 |

To run in pybmds, you'll first need to load the dataset, and create a `DichotomousDataset` object for each row:

In [ ]:
import os
from copy import deepcopy

import pandas as pd

import pybmds
from pybmds.models import dichotomous


# change output file to where you want it to belong
OUTPUT_FN = 'output/batch.xlsx'

# change input file to where you want it to belong
fn = 'data/batch.csv'
assert os.path.exists(fn)

df = pd.read_csv(fn)

def dichotomous_dictify(d):
    try:
        return dict(
              id=d.ID,
              doses=list(map(float, d.Dose.split(';'))),
              ns=list(map(int, d.N.split(';'))),
              incidences=list(map(int, d.Incidence.split(';'))),
        )
    except:
        print(f'Row {d.ID} not included')
        return None

def get_bmds_dataset(dtype, dataset):
    if dtype == pybmds.constants.Dtype.CONTINUOUS:
        cls = pybmds.ContinuousDataset
    elif dtype == pybmds.constants.Dtype.CONTINUOUS_INDIVIDUAL:
        cls = pybmds.ContinuousIndividualDataset
    elif dtype == pybmds.constants.Dtype.DICHOTOMOUS:
        cls = pybmds.DichotomousDataset
    return cls(**dataset)

dichotomous_data = [
    d for d in df.apply(dichotomous_dictify, axis=1)
    if d is not None
]

dichotomous_datasets = [
    get_bmds_dataset("D", data) for data in dichotomous_data
]

## Single model, multiple datasets

With datasets loaded, we can run a single model for each dataset if we'd like:

In [ ]:
dichotomous_results = []
for dataset in dichotomous_datasets:
    model = dichotomous.Multistage(
        dataset=dataset,
        settings={"degree": 2}
    )
    result = model.execute()
    dichotomous_results.append(model)

And then we could export a simple list of results:

In [ ]:
outputs = [
    [model.dataset.metadata.id, model.name(), model.results.bmd, model.results.bmdl, model.results.bmdu]
    for model in dichotomous_results
]
output_df = pd.DataFrame(data=outputs, columns="Dataset-ID Name BMD BMDL BMDU".split())
output_df.head()

## Session batch execution

Alternatively, you could run an BMDS modeling session, which executes a suite of models and returns the best fitting result.

In [ ]:
# function takes a dataset as input and returns an execution response
def runner(ds):
    sess = pybmds.Session(dataset=ds)
    sess.add_model(pybmds.Models.Logistic, settings={"bmr": 0.2})
    sess.add_model(pybmds.Models.Probit, settings={"bmr": 0.2})
    sess.execute_and_recommend()
    return pybmds.BatchResponse(success=True, content=[sess.to_dict()])

# execute all datasets and sessions on a single processor
batch = pybmds.BatchSession().execute(dichotomous_datasets, runner, nprocs=1)

Save Excel and Word reports:

In [ ]:
batch.to_excel("output/batch.xlsx")
batch.to_docx().save("output/batch.docx")

You could even run two sessions for each dataset, for example, running two different BMRs. The only change in the code above is modifying the `runner` function:

In [ ]:
def runner2(ds):
    sess1 = pybmds.Session(dataset=ds)
    sess1.add_model(pybmds.Models.Logistic, settings={"bmr": 0.1})
    sess1.add_model(pybmds.Models.Probit, settings={"bmr": 0.1})
    sess1.execute_and_recommend()

    sess2 = pybmds.Session(dataset=ds)
    sess2.add_model(pybmds.Models.Logistic, settings={"bmr": 0.2})
    sess2.add_model(pybmds.Models.Probit, settings={"bmr": 0.2})
    sess2.execute_and_recommend()

    return pybmds.BatchResponse(success=True, content=[sess1.to_dict(), sess2.to_dict()])

batch = pybmds.BatchSession().execute(dichotomous_datasets, runner2, nprocs=1)